In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

In [2]:
gdp_df=pd.read_csv('../data/GDP_byZip.csv')
taxes_df=pd.read_csv('../data/final_TAXES_by_zip.csv')